In [1]:
import os
from copy import deepcopy
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score
from sklearn.preprocessing import StandardScaler
from scipy.optimize import linear_sum_assignment
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

from duped_modules.dcn_duped import DCNDuped
from duped_modules.dec_duped import DECDuped, IDECDuped
from all_to_all_autoencoder import AllToAllAutoencoder
from all_to_cat_autoencoder import AllToCatAutoencoder
from cat_to_cat_autoencoder import CatToCatAutoencoder
from load_datasets import load_all_datasets

In [2]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
np.random.seed(0)

In [3]:
def cluster_accuracy(labels_true, labels_pred):
    # We need to map the labels to our cluster labels
    # This is a linear assignment problem on a bipartite graph
    k = max(len(np.unique(labels_true)), len(np.unique(labels_pred)))
    cost_matrix = np.zeros((k, k))
    for i in range(labels_true.shape[0]):
        cost_matrix[labels_true[i], labels_pred[i]] += 1
    inverted_cost_matrix = cost_matrix.max() - cost_matrix
    row_ind, col_ind = linear_sum_assignment(inverted_cost_matrix)
    return cost_matrix[row_ind, col_ind].sum() / labels_pred.size

In [77]:
def build_block(layers: list, activation_fn: torch.nn.Module = nn.LeakyReLU, output_fn: torch.nn.Module = nn.LeakyReLU, 
                bias: bool = True, batch_norm: bool = False, dropout: float = None):
    block_list = []
    for i in range(len(layers) - 1):
        block_list.append(torch.nn.Linear(layers[i], layers[i + 1], bias=bias))
        if batch_norm:
            block_list.append(torch.nn.BatchNorm1d(layers[i + 1]))
        if dropout is not None:
            block_list.append(torch.nn.Dropout(dropout))
        if activation_fn is not None:
            if (i != len(layers) - 2):
                block_list.append(activation_fn())
            else:
                if output_fn is not None:
                    block_list.append(output_fn())
    return torch.nn.Sequential(*block_list)

def build_autoencoder(input_dim: int, output_dim: int, layer_per_block: int, activation_fn: torch.nn.Module = nn.LeakyReLU, 
                      output_fn: torch.nn.Module = nn.LeakyReLU, bias: bool = True, batch_norm: bool = False, dropout: float = None):
    hidden_dim = min(input_dim//2, output_dim//2)

    encoder_layer_list = list(range(input_dim, hidden_dim - 1, min(-1, -round((input_dim - hidden_dim) / layer_per_block))))
    encoder_layer_list[-1] = hidden_dim
    encoder = build_block(encoder_layer_list, activation_fn, output_fn, bias, batch_norm, dropout)

    decoder_layer_list = list(range(hidden_dim, output_dim + 1, max(1, round((output_dim - hidden_dim) / layer_per_block))))
    decoder_layer_list[-1] = output_dim
    decoder = build_block(decoder_layer_list, activation_fn, output_fn, bias, batch_norm, dropout)

    return encoder, decoder

In [4]:
datasets = load_all_datasets(max_rows=5000)

accuracies = {d.name: {} for d in datasets}
nmis = {d.name: {} for d in datasets}

In [5]:
for d in datasets:
    print(f"{d.name}: Input dim: {d.input_dim}; Cat dim: {d.cat_dim}; Cont dim: {d.cont_dim}")

Abalone: Input dim: 9; Cat dim: 2; Cont dim: 7
Auction Verification: Input dim: 13; Cat dim: 12; Cont dim: 1
Bank Marketing: Input dim: 22; Cat dim: 17; Cont dim: 5
Breast Cancer: Input dim: 45; Cat dim: 45; Cont dim: 0
Census Income: Input dim: 58; Cat dim: 52; Cont dim: 6
Credit Approval: Input dim: 28; Cat dim: 22; Cont dim: 6
Heart Disease: Input dim: 17; Cat dim: 11; Cont dim: 6
Soybean Disease: Input dim: 55; Cat dim: 55; Cont dim: 0


In [6]:
abalone_encoder = nn.Sequential(
    nn.Linear(datasets.abalone.input_dim, 6),
    nn.LeakyReLU(),
    nn.Linear(6, 4),
    nn.LeakyReLU(),
)
abalone_decoder = nn.Sequential(
    nn.Linear(4, 6),
    nn.LeakyReLU(),
    nn.Linear(6, datasets.abalone.input_dim),
    nn.LeakyReLU(),
)

device = "cpu"
ae = AllToAllAutoencoder(deepcopy(abalone_encoder), deepcopy(abalone_decoder), datasets.abalone.input_dim, datasets.abalone.cat_dim, datasets.abalone.embedding_sizes, device=device)
ae.fit(datasets.abalone.dataloader, n_epochs=100, lr=0.001)

cat = torch.tensor(datasets.abalone.df[datasets.abalone.cat_cols].values, dtype=torch.int).detach().to(device)
cont = torch.tensor(datasets.abalone.df[datasets.abalone.cont_cols].values, dtype=torch.float).detach().to(device)
features = ae.encode(cat, cont).detach().cpu().numpy()
kmeans = KMeans(n_clusters=datasets.abalone.n_targets, init="random", n_init=10, max_iter=300, random_state=0, algorithm="lloyd").fit(features)

nmis[datasets.abalone.name]["All cols AE"] = normalized_mutual_info_score(datasets.abalone.y, kmeans.labels_)
accuracies[datasets.abalone.name]["All cols AE"] = cluster_accuracy(datasets.abalone.y, kmeans.labels_)

Epoch 25/100 - Batch Reconstruction loss: 0.124038
Epoch 50/100 - Batch Reconstruction loss: 0.106586
Epoch 75/100 - Batch Reconstruction loss: 0.125853
Epoch 100/100 - Batch Reconstruction loss: 0.182182


In [7]:
abalone_cat_encoder = nn.Sequential(
    nn.Linear(datasets.abalone.cat_dim, 2),
    nn.LeakyReLU(),
    nn.Linear(2, 1),
    nn.LeakyReLU(),
)
abalone_cat_decoder = nn.Sequential(
    nn.Linear(1, 2),
    nn.LeakyReLU(),
    nn.Linear(2, datasets.abalone.cat_dim),
    nn.LeakyReLU(),
)

device = "cpu"
ae = CatToCatAutoencoder(deepcopy(abalone_cat_encoder), deepcopy(abalone_cat_decoder), datasets.abalone.input_dim, datasets.abalone.cat_dim, datasets.abalone.embedding_sizes, device=device)
ae.fit(datasets.abalone.dataloader, n_epochs=100, lr=0.001)

cat = torch.tensor(datasets.abalone.df[datasets.abalone.cat_cols].values, dtype=torch.int).detach().to(device)
cont = datasets.abalone.df[datasets.abalone.cont_cols].values
features = np.concatenate((ae.encode(cat, cont).detach().cpu().numpy(), cont), axis=1)
kmeans = KMeans(n_clusters=datasets.abalone.n_targets, init="random", n_init=10, max_iter=300, random_state=0, algorithm="lloyd").fit(features)

nmis[datasets.abalone.name]["Cat Cols AE"] = normalized_mutual_info_score(datasets.abalone.y, kmeans.labels_)
accuracies[datasets.abalone.name]["Cat Cols AE"] = cluster_accuracy(datasets.abalone.y, kmeans.labels_)

Epoch 25/100 - Batch Reconstruction loss: 0.397039
Epoch 50/100 - Batch Reconstruction loss: 0.392506
Epoch 75/100 - Batch Reconstruction loss: 0.028997
Epoch 100/100 - Batch Reconstruction loss: 0.020927


In [12]:
abalone_all_to_cat_encoder = nn.Sequential(
    nn.Linear(datasets.abalone.input_dim, 6),
    nn.LeakyReLU(),
    nn.Linear(6, 3),
    nn.LeakyReLU(),
    nn.Linear(3, 1),
    nn.LeakyReLU(),
)
abalone_all_to_cat_decoder = nn.Sequential(
    nn.Linear(1, 2),
    nn.LeakyReLU(),
    nn.Linear(2, datasets.abalone.cat_dim),
    nn.LeakyReLU(),
)


device = "cpu"
ae = AllToCatAutoencoder(deepcopy(abalone_all_to_cat_encoder), deepcopy(abalone_all_to_cat_decoder), datasets.abalone.input_dim, datasets.abalone.cat_dim, datasets.abalone.embedding_sizes, device=device)
ae.fit(datasets.abalone.dataloader, n_epochs=100, lr=0.001)

cat = torch.tensor(datasets.abalone.df[datasets.abalone.cat_cols].values, dtype=torch.int).detach().to(device)
cont = torch.tensor(datasets.abalone.df[datasets.abalone.cont_cols].values, dtype=torch.float).detach().to(device)
features = np.concatenate((ae.encode(cat, cont).detach().cpu().numpy(), cont.cpu().numpy()), axis=1)
kmeans = KMeans(n_clusters=datasets.abalone.n_targets, init="random", n_init=10, max_iter=300, random_state=0, algorithm="lloyd").fit(features)

nmis[datasets.abalone.name]["All cols to cat cols AE"] = normalized_mutual_info_score(datasets.abalone.y, kmeans.labels_)
accuracies[datasets.abalone.name]["All cols to cat cols AE"] = cluster_accuracy(datasets.abalone.y, kmeans.labels_)

Epoch 25/100 - Batch Reconstruction loss: 0.037933
Epoch 50/100 - Batch Reconstruction loss: 0.023870
Epoch 75/100 - Batch Reconstruction loss: 0.004308
Epoch 100/100 - Batch Reconstruction loss: 0.001753


In [13]:
pd.DataFrame(nmis.values(), index=nmis.keys())

,All cols AE,Cat Cols AE,All cols to cat cols AE
Abalone,0.159608,0.17491,0.171676
Auction Verification,NaN,NaN,NaN
Bank Marketing,NaN,NaN,NaN
Breast Cancer,NaN,NaN,NaN
Census Income,NaN,NaN,NaN
Credit Approval,NaN,NaN,NaN
Heart Disease,NaN,NaN,NaN
Soybean Disease,NaN,NaN,NaN


In [14]:
pd.DataFrame(accuracies.values(), index=accuracies.keys())

,All cols AE,Cat Cols AE,All cols to cat cols AE
Abalone,0.136701,0.152262,0.148671
Auction Verification,NaN,NaN,NaN
Bank Marketing,NaN,NaN,NaN
Breast Cancer,NaN,NaN,NaN
Census Income,NaN,NaN,NaN
Credit Approval,NaN,NaN,NaN
Heart Disease,NaN,NaN,NaN
Soybean Disease,NaN,NaN,NaN
